## Data Parsing

In [1]:
import os
import re
import time
import json
import httpx
import random
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
lmap = lambda funcion, iterable: list(map(funcion, iterable))
lfilter = lambda funcion, iterable: list(filter(funcion, iterable))

In [3]:
data_dir = "../data/"
raw_dir = os.path.join(data_dir, "raw")
raw_csv_dir = os.path.join(raw_dir, "csv")
raw_json_dir = os.path.join(raw_dir, "json")
html_dir = os.path.join(raw_dir, "html")
completed_html_dir = os.path.join(html_dir, "completed")
upcoming_html_dir = os.path.join(html_dir, "upcoming")
fighterlist_html_dir = os.path.join(html_dir, "fighterlist")
fighters_html_dir = os.path.join(html_dir, "fighters")
completed_eventlist_html_dir = os.path.join(completed_html_dir, "eventlist")
completed_events_html_dir = os.path.join(completed_html_dir, "events")
completed_fights_html_dir = os.path.join(completed_html_dir, "fights")
upcoming_eventlist_html_dir = os.path.join(upcoming_html_dir, "eventlist")
upcoming_events_html_dir = os.path.join(upcoming_html_dir, "events")
upcoming_fights_html_dir = os.path.join(upcoming_html_dir, "fights")

In [4]:
dirs = [raw_csv_dir, raw_json_dir, fighters_html_dir, fighterlist_html_dir,
        completed_eventlist_html_dir, 
        completed_events_html_dir,
        upcoming_eventlist_html_dir, upcoming_events_html_dir,
       completed_fights_html_dir, upcoming_fights_html_dir]

for folderpath in dirs:
    os.makedirs(folderpath, exist_ok=True)

### Parse Fight page

In [5]:
def indiv_fight_data_extractor(fight_id_html):
    fight_id, fight_html = fight_id_html 
    fight_dict = {}

    soup = BeautifulSoup(fight_html)
    title_a_elem = soup.find("h2", class_="b-content__title").find("a")
    fight_dict["Event Name"] = title_a_elem.text.strip()
    fight_dict["Event Url"] = title_a_elem["href"]
    fight_dict["Fight ID"] = fight_id
    fighters = soup.find_all("div", class_="b-fight-details__person")
    for idx, fighter in enumerate(fighters, start=1):
        fight_dict[f"Fighter{idx} Status"] = fighter.find("i", class_="b-fight-details__person-status").text.strip()

        a_elem = fighter.find("a", class_="b-link b-fight-details__person-link")
        fight_dict[f"Fighter{idx} Name"] = a_elem.text.strip()
        fight_dict[f"Fighter{idx} Url"] = a_elem["href"]

    fight_dict["Bout"] = soup.find("i", class_="b-fight-details__fight-title").text.strip()

    fight_details_div = soup.find("div", class_="b-fight-details__content")

    method_elem = fight_details_div.find("p").find("i", class_="b-fight-details__text-item_first")

    def get_details(i_elem):
        i_text = i_elem.text.replace("\n", "").strip()
        m = re.search(r"(.*):\s+(.*)", i_text)
        if m:
            return (m.group(1), m.group(2))

    label_elems = fight_details_div.find_all("i", class_="b-fight-details__label")
    detail_elems = lmap(lambda e: e.parent, label_elems)
    detail_tups = lfilter(lambda t: t != None, map(get_details, detail_elems))

    for label, text in detail_tups:
        fight_dict[label] = text

    fight_dict

    details_text = fight_details_div.find_all("p")[1].text.replace("\n", "").strip()

    m = re.search(r"(.*):\s+(.*)", details_text)
    if m:
        fight_dict[m.group(1)] = m.group(2)

    fight_dict

    tables = soup.find_all("table")

    len(tables)

    def fight_tables_to_dicts(page_html: str):

        def extract_table(tables):
            data_dict = {key:[] for key in map(lambda x: x.text.strip(), tables[0].find("thead").find_all("th"))}
            data_dict["Round"] = []

            for i, table in enumerate(tables[:2]):
                rows = table.find("tbody").find_all("tr")

                for j, row in enumerate(rows, start=1):
                    for col, elem in zip(data_dict.keys(), row.find_all("td")):
                        if col == "Fighter":
                            for a_elem in elem.find_all("a"):
                                data_dict[col].append(a_elem.text.strip())
                        else:
                            for p_elem in elem.find_all("p"):
                                val = p_elem.text.strip()
                                val = val if val != "---" else None
                                data_dict[col].append(val)

                    if i == 0:
                        data_dict["Round"].extend(["Overall", "Overall"])
                    else:
                        data_dict["Round"].extend([f"Round {j}", f"Round {j}"])


            return data_dict


        soup = BeautifulSoup(page_html, features="lxml")
        tables = soup.find_all("table")
        
        return extract_table(tables[:2]), extract_table(tables[2:])
         

    try:
        fight_dict["Totals"], fight_dict["Significant Strikes"] = fight_tables_to_dicts(fight_html)
    except IndexError:
        print("Table IndexError")
        pass 
    
    return fight_dict

In [6]:
def all_fight_data_extractor(fights_html_dir):
    fights_html_dict = {}

    fight_files = lfilter(lambda x: x.endswith(".html"), os.listdir(fights_html_dir))
    for filename in tqdm(fight_files):
        filepath = os.path.join(fights_html_dir, filename)
        with open(filepath, "r") as f:
            html_str = f.read()
            fight_id = filename.replace(".html","")
            fights_html_dict[fight_id] = html_str

    # fights_dict_list = []
    
    # for fight_id, fight_html in tqdm(fights_html_dict.items()):
    #     fights_dict_list.append(indiv_fight_data_extractor(fight_id, fight_html))
    
    with Pool(16) as p:
        fights_dict_list = p.map(indiv_fight_data_extractor, fights_html_dict.items())
        
    return fights_dict_list

In [157]:
completed_fights_dict_list = all_fight_data_extractor(completed_fights_html_dir)

100%|██████████| 6875/6875 [00:00<00:00, 26644.24it/s]


Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError
Table IndexError


In [158]:
len(completed_fights_dict_list)

6875

In [159]:
filepath = os.path.join(raw_json_dir, "completed_fights_data.json")
with open(filepath, "w") as f:
    json.dump(completed_fights_dict_list, f, indent=4)

### Parse Fighter pages

In [7]:
def extract_fighter_data(fighter_page_filepath):

    with open(fighter_page_filepath, "r") as f:
        html_str = f.read()

    soup = BeautifulSoup(html_str)

    fighter_dict = dict()

    fighter_dict["Name"] = soup.find("span", class_="b-content__title-highlight").text.strip() 

    fighter_dict["Fighter ID"] = os.path.split(fighter_page_filepath)[1].replace(".html","")
    
    fighter_dict["Record"] = soup.find("span", class_="b-content__title-record").text.replace("Record:","").strip() 

    nn = soup.find("p", class_="b-content__Nickname").text.strip()
    fighter_dict["Nickname"] = nn if nn else None

    fighter_dict

    uls = soup.find_all("ul", class_="b-list__box-list")

    info_list = [li.text.replace("\n","").strip() for ul in uls for li in ul.find_all("li")]

    info_list = lfilter(lambda s: s, info_list)

    for info in info_list:
        m = re.search(r"(.*):\s+(.*)", info)
        if m:
            key, val = m.group(1).strip(), m.group(2).strip()
            if key == "Height":
                val = val.replace(" ","")
            if val == "--":
                val = None
            fighter_dict[key] = val

    return fighter_dict

In [8]:
def create_fighters_df(fighters_html_dir):
    fighter_files = os.listdir(fighters_html_dir)
    fighter_files = lfilter(lambda s: s.endswith(".html"), fighter_files)
    fighter_filepaths = lmap(lambda s: os.path.join(fighters_html_dir, s), fighter_files)
    
    with Pool(16) as p:
        fighters_lod = p.map(extract_fighter_data, fighter_filepaths)
        
    return pd.DataFrame(fighters_lod)

In [9]:
fighters_df = create_fighters_df(fighters_html_dir)

In [10]:
fighters_df

,Name,Fighter ID,Record,Nickname,Height,Weight,Reach,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def,TD Avg.,TD Acc.,TD Def.,Sub. Avg.
0,Tamires Vidal,4f3064faf17aa711,7-1-0,Tractor,"5'6""",135 lbs.,"68""",Orthodox,"Jul 27, 1998",5.48,58%,4.19,7%,0.00,0%,0%,0.0
1,Tatsuya Kawajiri,80d918336163b80c,36-11-2,Crusher,"5'7""",145 lbs.,"69""",Orthodox,"May 08, 1978",2.04,44%,2.01,50%,3.63,48%,63%,1.1
2,Joe Giannetti,946db8a378903cc3,6-1-0,None,"6'0""",155 lbs.,"74""",Southpaw,"Jul 05, 1995",1.93,38%,2.80,40%,1.00,16%,0%,0.0
3,Brock Jardine,64959f90b6dd9759,12-6-0,The Machine,"5'11""",170 lbs.,None,Orthodox,"Jul 10, 1985",1.06,30%,3.34,56%,0.00,0%,14%,0.8
4,Melissa Martinez,3f4c3bc822bea45d,7-1-0,Super Mely,"5'2""",115 lbs.,"66""",Southpaw,"Aug 18, 1997",3.00,57%,2.53,45%,1.00,50%,0%,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2384,Travis Fulton,911fb265462f0d94,253-53-10 (1 NC),The Ironman,"6'0""",240 lbs.,None,Orthodox,"May 29, 1977",0.00,0%,0.00,0%,0.00,0%,0%,0.0
2385,Henry Cejudo,056c493bbd76a918,16-2-0,The Messenger,"5'4""",135 lbs.,"64""",Orthodox,"Feb 09, 1987",3.92,45%,2.77,65%,2.12,33%,93%,0.2
2386,Ross Pointon,7215953cd5c6254d,6-17-0,The Gladiator,"5'8""",170 lbs.,None,Orthodox,"Feb 18, 1980",2.62,23%,2.77,58%,1.19,100%,0%,0.0
2387,Benoit Saint Denis,c2299ec916bc7c56,10-1-0 (1 NC),God of War,"5'11""",155 lbs.,"73""",Southpaw,"Dec 18, 1995",4.37,52%,6.72,41%,3.36,30%,85%,1.1


In [11]:
filepath = os.path.join(raw_csv_dir, "fighters_data.csv")
fighters_df.to_csv(filepath, index=False)